# Import Data


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import pandas as pd

df = pd.read_excel('news.xlsx')
df

#Preprocessing

## Hapus Missing Value

In [ ]:
!pip install fancyimpute

In [ ]:
import pandas as pd
from fancyimpute import IterativeImputer

positivity = df['positivity']
imputer = IterativeImputer(max_iter=10, random_state=0)
positivity_imputed = imputer.fit_transform(positivity.values.reshape(-1, 1))


In [ ]:
df['positivity'] = positivity_imputed
df

In [ ]:
# Menghitung jumlah missing value per kolom
missing_values = df.isna().sum()

# Menampilkan hasil
print(missing_values)

In [ ]:
df = df.drop(columns=['positivity_gold'])
df = df.drop(columns=['relevance_gold'])
df = df.drop(columns=['Unnamed: 15'])
df = df.drop(columns=['Unnamed: 16'])
df = df.drop(columns=['Unnamed: 17'])
df = df.drop(columns=['Unnamed: 18'])
df = df.drop(columns=['Unnamed: 19'])
df = df.drop(columns=['Unnamed: 20'])
df = df.drop(columns=['Unnamed: 21'])
df = df.drop(columns=['Unnamed: 22'])
df = df.drop(columns=['Unnamed: 23'])

In [ ]:
mode_value = df['text'].mode()[0] #pake modus
df['text'].fillna(mode_value, inplace=True)


In [ ]:
import nltk
from nltk import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
import re
from textblob import TextBlob

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
!pip install PySastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
# Preprocessing
text = df["text"].to_string()
text

## Menggunakan Stopwords dan Stemming

In [ ]:
# Menghapus Stopword
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

stop = stopword.remove(text.lower())

# Stemming
stemmer = StemmerFactory().create_stemmer()
stem = stemmer.stem(stop)

## Hapus Karakter Tertentu

In [ ]:
def rm_punct2(text):
    return re.sub(
        r'[\!\"\#\$\%\&\'\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~]',
        ' ', text)

def rm_html(text):
    return re.sub(r'<[^>]+>', '', text)

def rm_number(text):
    return re.sub(r'\d+', '', text)

def rm_whitespaces(text):
    return re.sub(r' +', ' ', text)

def rm_nonascii(text):
    return re.sub(r'[^\x00-\x7f]', r'', text)

def rm_emoji(text):
    emojis = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE
    )
    return emojis.sub(r'', text)

def spell_correction(text):
    return re.sub(r'(.)\1+', r'\1\1', text)

def clean_pipeline(text):
    no_html = rm_html(text)
    no_punct = rm_punct2(no_html)
    no_number = rm_number(no_punct)
    no_whitespaces = rm_whitespaces(no_number)
    no_nonasci = rm_nonascii(no_whitespaces)
    no_emoji = rm_emoji(no_nonasci)
    spell_corrected = spell_correction(no_emoji)
    clean_lowered = spell_corrected.lower()
    return clean_lowered

cleaned = clean_pipeline(stem)
cleaned

## Tokenisasi Menggunakan TextBlob

In [ ]:
# Tokenisasi kata dengan TextBlob
tokens = TextBlob(cleaned).words
print(tokens)

fdist = FreqDist(tokens)
print(fdist)

fdist1 = fdist.most_common(10)
print(fdist1)

## Mengganti Slang & Kata Singkatan

In [ ]:
abbreviation_dict = {
    "abt": "about",
    "afk": "away from keyboard",
    "asap": "as soon as possible",
    "bday": "birthday",
    "brb": "be right back",
    "btw": "by the way",
    "cya": "see you",
    "diy": "do it yourself",
    "fwiw": "for what it's worth",
    "fyi": "for your information",
    "gr8": "great",
    "gtg": "got to go",
    "idk": "I don't know",
    "irl": "in real life",
    "lmao": "laughing my ass off",
    "lol": "laugh out loud",
    "np": "no problem",
    "omg": "oh my god",
    "plz": "please",
    "rofl": "rolling on the floor laughing",
    "smh": "shaking my head",
    "thx": "thanks",
    "tmi": "too much information",
    "ttyl": "talk to you later",
    "wtf": "what the f***",
    "yolo": "you only live once",
}


In [ ]:
expanded_text = []
for token in tokens:
    expanded_text.append(abbreviation_dict.get(token.lower(), token))

cleaned_text = " ".join(expanded_text)

## Visualisasi Hasil Preprocessing Menggunakan Wordcloud

In [ ]:
from matplotlib import pyplot as plt
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer

# Menghitung bobot TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform([' '.join(expanded_text)])

# Membuat Word Cloud
wordcloud = WordCloud(width=800, height=400, background_color='white')
wordcloud.generate_from_frequencies({kata: bobot for kata, bobot in
                                     zip(tfidf_vectorizer.get_feature_names_out(),
                                         tfidf_matrix.toarray()[0])})

# Menampilkan Word Cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

#Term Weighting menggunakan Word Embedding (Word2Vec)

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize


In [ ]:
# importing all necessary modules
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

warnings.filterwarnings(action = 'ignore')

import gensim
from gensim.models import Word2Vec


# Replaces escape character with space
f = cleaned_text.replace("\n", " ")

data = []

# iterate through each sentence in the file
for i in sent_tokenize(f):
    temp = []

    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())

    data.append(temp)

# Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count = 1,
                              vector_size = 100, window = 5)

# Print results
print("Cosine similarity between 'the' " +
               "and 'administration' - CBOW : ",
    model1.wv.similarity('the', 'administration'))

# Create Skip Gram model
model2 = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100,
                                             window = 5, sg = 1)

# Print results
print("Cosine similarity between 'the' " +
          "and 'administration' - Skip Gram : ",
    model2.wv.similarity('the', 'administration'))


# Classification

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
text = list(df['text'])

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
corpus = []

for i in range(len(text)):
    text[i] = str(text[i])
    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('english')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)

#assign corpus to data['text']

df['text'] = corpus

df.head()


# Logistic Regression

## Train Test Split

In [ ]:
# Create Feature and Label sets

X = df['text']
y = df['relevance']

# train test split (66% train - 33% test)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)
print('Training Data :', X_train.shape)
print('Testing Data : ', X_test.shape)

## Feature Extraction

In [ ]:
# Train Bag of Words model

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

X_train_cv = cv.fit_transform(X_train)

X_train_cv.shape

## Model Training & Evaluation

In [ ]:
# Training Logistic Regression model

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train_cv, y_train)


# transform X_test using CV

X_test_cv = cv.transform(X_test)


# generate predictions

predictions = lr.predict(X_test_cv)

predictions

## Confusion Matrix

In [ ]:
# confusion matrix

import pandas as pd

from sklearn import metrics

df = pd.DataFrame(metrics.confusion_matrix(y_test,predictions), index=['yes','no', 'no'], columns=['no','no','no'])

df

#Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train_cv, y_train)
rf_predictions = rf.predict(X_test_cv)


## Accuracy Score

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Menghitung akurasi
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Classifier Accuracy:", rf_accuracy)



## Confusion Matrix

In [ ]:
# Confusion matrix
rf_confusion_matrix = confusion_matrix(y_test, rf_predictions)
print("Confusion Matrix for Random Forest Classifier:")
print(rf_confusion_matrix)

# Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train_cv, y_train)
nb_predictions = nb.predict(X_test_cv)


## Accuracy Score

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Menghitung akurasi model
accuracy = accuracy_score(y_test, nb_predictions)
print("Akurasi Model Naive Bayes:", accuracy)


In [ ]:
# Membuat confusion matrix
confusion = confusion_matrix(y_test, nb_predictions)
print("Confusion Matrix Model Naive Bayes:")
print(confusion)

# Support Vector Machine (SVM) Classifier

In [ ]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(X_train_cv, y_train)
svm_predictions = svm.predict(X_test_cv)


## Accuracy Score

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Hitung akurasi
accuracy = accuracy_score(y_test, svm_predictions)
print("Accuracy: {:.2f}%".format(accuracy * 100))


## Confusion Matrix

In [ ]:

# Hitung confusion matrix
conf_matrix = confusion_matrix(y_test, svm_predictions)
print("Confusion Matrix:")
print(conf_matrix)

# K-Nearest Neighbors (KNN) Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train_cv, y_train)
knn_predictions = knn.predict(X_test_cv)


## Accuracy Score

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Menghitung akurasi
knn_accuracy = accuracy_score(y_test, knn_predictions)
print("K-Nearest Neighbors (KNN) Accuracy:", knn_accuracy)




## Confusion Matrix

In [ ]:
# Membuat confusion matrix
knn_confusion_matrix = confusion_matrix(y_test, knn_predictions)
print("K-Nearest Neighbors (KNN) Confusion Matrix:")
print(knn_confusion_matrix)